In [ ]:
import plotly.graph_objs as go
 
def update_graphs():
    global buffer_data
    # Convert buffer_data to separate lists for X, Y, Z
    x_data = [data[0] for data in buffer_data]
    y_data = [data[1] for data in buffer_data]
    z_data = [data[2] for data in buffer_data]
    # Plot the data
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=list(range(len(x_data))), y=x_data, mode='lines', name='X Axis'))
    fig.add_trace(go.Scatter(x=list(range(len(y_data))), y=y_data, mode='lines', name='Y Axis'))
    fig.add_trace(go.Scatter(x=list(range(len(z_data))), y=z_data, mode='lines', name='Z Axis'))
    # Save the plot
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    fig.write_image(f'plot_{timestamp}.png')
    # Save the data to CSV
    with open(f'data_{timestamp}.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['X', 'Y', 'Z'])
        writer.writerows(buffer_data)
    # Update the Plotly Dash app
    # Assuming `app` is your Dash app and `graph_id` is the ID of the graph component
    app.update_layout(
        go.Figure(data=[
            go.Scatter(x=list(range(len(x_data))), y=x_data, mode='lines', name='X Axis'),
            go.Scatter(x=list(range(len(y_data))), y=y_data, mode='lines', name='Y Axis'),
            go.Scatter(x=list(range(len(z_data))), y=z_data, mode='lines', name='Z Axis')
        ])
    )

In [ ]:
from collections import deque
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
from datetime import datetime
import csv
from arduino_iot_cloud import ArduinoCloudClient


# Arduino Cloud Configuration
DEVICE_ID = "0629dcd4-8bd9-4b74-abb0-07954884af79"
SECRET_KEY = "Z!UrpQKrv4JzYInsw6NRRiTvw"

# Global variables for sensor data
x_data = []
y_data = []
z_data = []

def create_live_update_dash(data_fetcher, graph_id='live-graph', refresh_rate=1000, max_samples=200, port=8050):
    
    # Buffer to hold incoming data points
    data_buffer = deque(maxlen=max_samples)
 
    # Initialize Dash app
    app = dash.Dash(__name__)
    app.layout = html.Div([
        dcc.Graph(id=graph_id),
        dcc.Interval(id='interval-component', interval=refresh_rate, n_intervals=0)
    ])
 
    @app.callback(
        Output(graph_id, 'figure'),
        [Input('interval-component', 'n_intervals')]
    )
    def update_graph(n):
        # Fetch new data from the data source
        new_data = data_fetcher()
 
        if new_data:
            data_buffer.append(new_data)  # Add new data to the buffer
 
        # Extract X, Y, Z components from the buffer
        x_data = [point[0] for point in data_buffer]
        y_data = [point[1] for point in data_buffer]
        z_data = [point[2] for point in data_buffer]
 
        # Create the figure for the graph
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=list(range(len(x_data))), y=x_data, mode='lines+markers', name='X Axis'))
        fig.add_trace(go.Scatter(x=list(range(len(y_data))), y=y_data, mode='lines+markers', name='Y Axis'))
        fig.add_trace(go.Scatter(x=list(range(len(z_data))), y=z_data, mode='lines+markers', name='Z Axis'))
 
        # Set graph layout
        fig.update_layout(
            title='Live Accelerometer Data',
            xaxis=dict(title='Time'),
            yaxis=dict(title='Values'),
            uirevision='constant'  # Prevent auto-scaling during updates
        )
 
        return fig
 
    # Run the Dash server
    app.run_server(debug=True, port=port)

def real_data_fetcher():
    # Retrieve the most recent data
    if x_data and y_data and z_data:
        return [x_data[-1], y_data[-1], z_data[-1]]
    else:
        return None
    
# Example of how to use the reusable function with real data
create_live_update_dash(data_fetcher=real_data_fetcher, refresh_rate=500, max_samples=150, port=8060)
